In [17]:
import tensorflow as tf
import os
import cv2
from matplotlib import pyplot as plt
%matplotlib inline  

from datasets import flower_dataset
#from models import models
from backbone import inception
from preprocessing import inception_preprocessing

In [30]:
dataset_dir = '/home/dan/prj/datasets/flowers/flower_example1/train'

In [48]:
def load_categorical_data(dataset_dir):
  filepaths = {}
  class_names = []
  for dir_name in os.listdir(dataset_dir):
    directory = os.path.join(dataset_dir, dir_name)
    if os.path.isdir(directory):
        filepaths[dir_name] = []
        for filename in os.listdir(directory):
            path = os.path.join(directory, filename)
            filepaths[dir_name].append(path)
        class_names.append(dir_name)
  class_names_to_ids = dict(zip(class_names, range(len(class_names))))
  return filepaths, class_names_to_ids

In [49]:
filepaths, class_names_to_ids = load_categorical_data(dataset_dir)

In [21]:
batch_size = 10

In [51]:
def _parse_function(filename, label=None):
  image_string = tf.read_file(filename)
  image_decoded = tf.image.decode_jpeg(image_string, channels=3)
  # image_resized = tf.image.resize_images(image_decoded, [224, 224], tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  images = inception_preprocessing.preprocess_image(
      image_decoded, height=224, width=224, is_training=True)

  return images

In [52]:
iterator = {}
for name in class_names_to_ids.keys():
    dataset = tf.data.Dataset.from_tensor_slices(tf.cast(
        filepaths[name], tf.string))
    dataset = dataset.shuffle(len(filepaths[name]))
    dataset = dataset.repeat()
    dataset = dataset.map(_parse_function, num_parallel_calls=2)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(2 * batch_size)
    

    iterator[name] = dataset.make_one_shot_iterator()

In [53]:
images = iterator['roses'].get_next()

In [54]:
with tf.Session() as sess:
    real_images = sess.run(images)

In [56]:
real_images.shape

(10, 224, 224, 3)

In [68]:
dataset1 = tf.data.Dataset.from_tensor_slices( list(range(10)) )
print("dataset1 : ")
print(dataset1.output_types)
print(dataset1.output_shapes)

dataset2 = tf.data.Dataset.from_tensor_slices( list(range(10)))

print("dataset2 : ")
print(dataset2.output_types)
print(dataset2.output_shapes)

dataset3 = dataset1.concatenate(dataset2)

dataset1 : 
<dtype: 'int32'>
()
dataset2 : 
<dtype: 'int32'>
()


In [69]:
iterator1 = dataset3.make_one_shot_iterator()
x = iterator1.get_next()

In [70]:
with tf.Session() as sess:
    for i in range(20):
        print(sess.run(x))

[0 1]
[2 3]
[4 5]
[6 7]
[8 9]
[0 1]
[2 3]
[4 5]
[6 7]
[8 9]


OutOfRangeError: End of sequence
	 [[node IteratorGetNext_5 (defined at <ipython-input-69-c6ec242d6730>:2)  = IteratorGetNext[output_shapes=[[?]], output_types=[DT_INT32], _device="/job:localhost/replica:0/task:0/device:CPU:0"](OneShotIterator_14)]]

Caused by op 'IteratorGetNext_5', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/dan/.virtualenvs/tf/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/dan/.virtualenvs/tf/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/dan/.virtualenvs/tf/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/dan/.virtualenvs/tf/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 132, in star